<h1>8장 시맨틱 검색과 RAG</h1>
<i>LLM의 핵심 요소인 검색을 살펴 봅니다.</i>

<a href="https://github.com/rickiepark/handson-llm"><img src="https://img.shields.io/badge/GitHub%20Repository-black?logo=github"></a>
[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/rickiepark/handson-llm/blob/main/chapter07.ipynb)

---

이 노트북은 <[핸즈온 LLM](https://www.amazon.com/Hands-Large-Language-Models-Understanding/dp/1098150961)> 책 8장의 코드를 담고 있습니다.

---

<a href="https://www.amazon.com/Hands-Large-Language-Models-Understanding/dp/1098150961">
<img src="https://raw.githubusercontent.com/HandsOnLLM/Hands-On-Large-Language-Models/main/images/book_cover.png" width="350"/></a>

### [선택사항] - <img src="https://colab.google/static/images/icons/colab.png" width=100>에서 패키지 선택하기


이 노트북을 구글 코랩에서 실행한다면 다음 코드 셀을 실행하여 이 노트북에서 필요한 패키지를  설치하세요.

---

💡 **NOTE**: 이 노트북의 코드를 실행하려면 GPU를 사용하는 것이 좋습니다. 구글 코랩에서는 **런타임 > 런타임 유형 변경 > 하드웨어 가속기 > T4 GPU**를 선택하세요.

---

In [1]:
%%capture
!pip install cohere faiss-cpu rank_bm25 langchain-community

# 사용하는 파이썬과 CUDA 버전에 맞는 llama-cpp-python 패키지를 설치하세요.
# 현재 코랩의 파이썬 버전은 3.11이며 CUDA 버전은 12.4입니다.
!pip install https://github.com/abetlen/llama-cpp-python/releases/download/v0.3.4-cu124/llama_cpp_python-0.3.4-cp311-cp311-linux_x86_64.whl

## 밀집 검색 예제

### 1. 텍스트 문서 분할하기


In [2]:
import cohere

# 코히어의 API 키를 입력하세요.
api_key = ''

# 코히어 클라이언트를 만듭니다.
co = cohere.Client(api_key)

In [3]:
text = """
Interstellar is a 2014 epic science fiction film co-written, directed, and produced by Christopher Nolan.
It stars Matthew McConaughey, Anne Hathaway, Jessica Chastain, Bill Irwin, Ellen Burstyn, Matt Damon, and Michael Caine.
Set in a dystopian future where humanity is struggling to survive, the film follows a group of astronauts who travel through a wormhole near Saturn in search of a new home for mankind.

Brothers Christopher and Jonathan Nolan wrote the screenplay, which had its origins in a script Jonathan developed in 2007.
Caltech theoretical physicist and 2017 Nobel laureate in Physics[4] Kip Thorne was an executive producer, acted as a scientific consultant, and wrote a tie-in book, The Science of Interstellar.
Cinematographer Hoyte van Hoytema shot it on 35 mm movie film in the Panavision anamorphic format and IMAX 70 mm.
Principal photography began in late 2013 and took place in Alberta, Iceland, and Los Angeles.
Interstellar uses extensive practical and miniature effects and the company Double Negative created additional digital effects.

Interstellar premiered on October 26, 2014, in Los Angeles.
In the United States, it was first released on film stock, expanding to venues using digital projectors.
The film had a worldwide gross over $677 million (and $773 million with subsequent re-releases), making it the tenth-highest grossing film of 2014.
It received acclaim for its performances, direction, screenplay, musical score, visual effects, ambition, themes, and emotional weight.
It has also received praise from many astronomers for its scientific accuracy and portrayal of theoretical astrophysics. Since its premiere, Interstellar gained a cult following,[5] and now is regarded by many sci-fi experts as one of the best science-fiction films of all time.
Interstellar was nominated for five awards at the 87th Academy Awards, winning Best Visual Effects, and received numerous other accolades"""

# 문장을 나누어 리스트로 만듭니다.
texts = text.split('.')

# 공백과 줄바꿈 문자를 삭제합니다.
texts = [t.strip(' \n') for t in texts]

### 2. 문장을 임베딩하기


In [4]:
import numpy as np

# 임베딩을 만듭니다.
response = co.embed(
  texts=texts,
  input_type="search_document",
).embeddings

embeds = np.array(response)
print(embeds.shape)

(15, 4096)


### 3. 검색 인덱스 구축하기


In [5]:
import faiss

dim = embeds.shape[1]
index = faiss.IndexFlatL2(dim)
index.add(np.float32(embeds))

### 4. 인덱스 검색하기


In [6]:
import pandas as pd

def search(query, number_of_results=3):

  # 1. 쿼리의 임베딩을 만듭니다.
  query_embed = co.embed(texts=[query],
                input_type="search_query",).embeddings[0]

  # 2. 최근접 이웃을 추출합니다.
  distances, similar_item_ids = index.search(np.float32([query_embed]),
                                             number_of_results)

  # 3. 데이터프레임을 사용해 출력을 준비합니다.
  texts_np = np.array(texts) # 인덱싱을 쉽게 하기 위해 텍스트 리스트를 넘파이 배열로 변환합니다.
  results = pd.DataFrame(data={'텍스트': texts_np[similar_item_ids[0]],
                               '거리': distances[0]})

  # 4. 결과를 출력하고 반환합니다.
  print(f"쿼리:'{query}'\n최근접 이웃:")
  return results

In [7]:
query = "how precise was the science"
results = search(query)
results

쿼리:'how precise was the science'
최근접 이웃:


,텍스트,거리
0,It has also received praise from many astronom...,10757.365234
1,Caltech theoretical physicist and 2017 Nobel l...,11566.132812
2,Interstellar uses extensive practical and mini...,11922.839844


In [8]:
from rank_bm25 import BM25Okapi
from sklearn.feature_extraction import _stop_words
import string

def bm25_tokenizer(text):
    tokenized_doc = []
    for token in text.lower().split():
        token = token.strip(string.punctuation)

        if len(token) > 0 and token not in _stop_words.ENGLISH_STOP_WORDS:
            tokenized_doc.append(token)
    return tokenized_doc

In [9]:
from tqdm import tqdm

tokenized_corpus = []
for passage in tqdm(texts):
    tokenized_corpus.append(bm25_tokenizer(passage))

bm25 = BM25Okapi(tokenized_corpus)

100%|██████████| 15/15 [00:00<00:00, 22869.71it/s]


In [10]:
def keyword_search(query, top_k=3, num_candidates=15):
    print("입력 질문:", query)

    ##### BM25 검색 (어휘 검색) #####
    bm25_scores = bm25.get_scores(bm25_tokenizer(query))
    top_n = np.argpartition(bm25_scores, -num_candidates)[-num_candidates:]
    bm25_hits = [{'corpus_id': idx, 'score': bm25_scores[idx]} for idx in top_n]
    bm25_hits = sorted(bm25_hits, key=lambda x: x['score'], reverse=True)

    print(f"탑-3 어휘 검색 (BM25) 결과")
    for hit in bm25_hits[0:top_k]:
        print("\t{:.3f}\t{}".format(hit['score'], texts[hit['corpus_id']].replace("\n", " ")))

In [11]:
keyword_search(query = "how precise was the science")

입력 질문: how precise was the science
탑-3 어휘 검색 (BM25) 결과
	1.789	Interstellar is a 2014 epic science fiction film co-written, directed, and produced by Christopher Nolan
	1.373	Caltech theoretical physicist and 2017 Nobel laureate in Physics[4] Kip Thorne was an executive producer, acted as a scientific consultant, and wrote a tie-in book, The Science of Interstellar
	0.000	It stars Matthew McConaughey, Anne Hathaway, Jessica Chastain, Bill Irwin, Ellen Burstyn, Matt Damon, and Michael Caine


### 밀집 검색의 단점


In [12]:
query = "What is the mass of the moon?"
results = search(query)
results

쿼리:'What is the mass of the moon?'
최근접 이웃:


,텍스트,거리
0,Cinematographer Hoyte van Hoytema shot it on 3...,12854.442383
1,The film had a worldwide gross over $677 milli...,13301.006836
2,It has also received praise from many astronom...,13332.000000


## 리랭킹 예제


In [13]:
query = "how precise was the science"
results = co.rerank(query=query, documents=texts, top_n=3, return_documents=True)
results.results

[RerankResponseResultsItem(document=RerankResponseResultsItemDocument(text='It has also received praise from many astronomers for its scientific accuracy and portrayal of theoretical astrophysics'), index=12, relevance_score=0.16981852),
 RerankResponseResultsItem(document=RerankResponseResultsItemDocument(text='The film had a worldwide gross over $677 million (and $773 million with subsequent re-releases), making it the tenth-highest grossing film of 2014'), index=10, relevance_score=0.07004896),
 RerankResponseResultsItem(document=RerankResponseResultsItemDocument(text='Caltech theoretical physicist and 2017 Nobel laureate in Physics[4] Kip Thorne was an executive producer, acted as a scientific consultant, and wrote a tie-in book, The Science of Interstellar'), index=4, relevance_score=0.0043994132)]

In [14]:
for idx, result in enumerate(results.results):
    print(idx, result.relevance_score , result.document.text)

0 0.16981852 It has also received praise from many astronomers for its scientific accuracy and portrayal of theoretical astrophysics
1 0.07004896 The film had a worldwide gross over $677 million (and $773 million with subsequent re-releases), making it the tenth-highest grossing film of 2014
2 0.0043994132 Caltech theoretical physicist and 2017 Nobel laureate in Physics[4] Kip Thorne was an executive producer, acted as a scientific consultant, and wrote a tie-in book, The Science of Interstellar


In [15]:
def keyword_and_reranking_search(query, top_k=3, num_candidates=10):
    print("입력 질문:", query)

    ##### BM25 검색 (어휘 검색) #####
    bm25_scores = bm25.get_scores(bm25_tokenizer(query))
    top_n = np.argpartition(bm25_scores, -num_candidates)[-num_candidates:]
    bm25_hits = [{'corpus_id': idx, 'score': bm25_scores[idx]} for idx in top_n]
    bm25_hits = sorted(bm25_hits, key=lambda x: x['score'], reverse=True)

    print(f"탑-3 어휘 검색 (BM25) 결과")
    for hit in bm25_hits[0:top_k]:
        print("\t{:.3f}\t{}".format(hit['score'], texts[hit['corpus_id']].replace("\n", " ")))

    # 리랭킹 추가
    docs = [texts[hit['corpus_id']] for hit in bm25_hits]

    print(f"\n리랭킹으로 얻은 탑-3 결과 ({len(bm25_hits)}개의 BM25 결과를 재조정함)")
    results = co.rerank(query=query, documents=docs, top_n=top_k, return_documents=True)
    for hit in results.results:
        print("\t{:.3f}\t{}".format(hit.relevance_score, hit.document.text.replace("\n", " ")))

In [16]:
keyword_and_reranking_search(query = "how precise was the science")

입력 질문: how precise was the science
탑-3 어휘 검색 (BM25) 결과
	1.789	Interstellar is a 2014 epic science fiction film co-written, directed, and produced by Christopher Nolan
	1.373	Caltech theoretical physicist and 2017 Nobel laureate in Physics[4] Kip Thorne was an executive producer, acted as a scientific consultant, and wrote a tie-in book, The Science of Interstellar
	0.000	Interstellar uses extensive practical and miniature effects and the company Double Negative created additional digital effects

리랭킹으로 얻은 탑-3 결과 (10개의 BM25 결과를 재조정함)
	0.004	Caltech theoretical physicist and 2017 Nobel laureate in Physics[4] Kip Thorne was an executive producer, acted as a scientific consultant, and wrote a tie-in book, The Science of Interstellar
	0.004	Set in a dystopian future where humanity is struggling to survive, the film follows a group of astronauts who travel through a wormhole near Saturn in search of a new home for mankind
	0.003	Brothers Christopher and Jonathan Nolan wrote the screenplay, w

## RAG

### 예: LLM API를 사용한 근거 기반 생성


In [17]:
query = "income generated"

# 1- 검색
# 여기서는 임베딩 검색을 사용하지만 하이브리드 방식이 이상적입니다.
results = search(query)

# 2- 근거 기반 생성
docs_dict = [{'text': text} for text in results['텍스트']]
response = co.chat(
    message = query,
    documents=docs_dict
)

print(response.text)

쿼리:'income generated'
최근접 이웃:
The film had a worldwide gross of over $677 million, and $773 million with subsequent re-releases.


In [18]:
response

NonStreamedChatResponse(text='The film had a worldwide gross of over $677 million, and $773 million with subsequent re-releases.', generation_id='b496e8b9-9164-42a3-867a-f55b7e02251f', response_id='616b45b4-c32f-42f6-b38d-29653e37fa36', citations=[ChatCitation(start=15, end=98, text='worldwide gross of over $677 million, and $773 million with subsequent re-releases.', document_ids=['doc_0'], type='TEXT_CONTENT')], documents=[{'id': 'doc_0', 'text': 'The film had a worldwide gross over $677 million (and $773 million with subsequent re-releases), making it the tenth-highest grossing film of 2014'}], is_search_required=None, search_queries=None, search_results=None, finish_reason='COMPLETE', tool_calls=None, chat_history=[UserMessage(role='USER', message='income generated', tool_calls=None), ChatbotMessage(role='CHATBOT', message='The film had a worldwide gross of over $677 million, and $773 million with subsequent re-releases.', tool_calls=None)], prompt=None, meta=ApiMeta(api_version=Ap

In [19]:
response.citations

[ChatCitation(start=15, end=98, text='worldwide gross of over $677 million, and $773 million with subsequent re-releases.', document_ids=['doc_0'], type='TEXT_CONTENT')]

### 예: 로컬 모델을 사용한 RAG


#### 생성 모델 로드하기


In [20]:
!wget https://huggingface.co/microsoft/Phi-3-mini-4k-instruct-gguf/resolve/main/Phi-3-mini-4k-instruct-q4.gguf

--2025-02-10 07:31:08--  https://huggingface.co/microsoft/Phi-3-mini-4k-instruct-gguf/resolve/main/Phi-3-mini-4k-instruct-q4.gguf
Resolving huggingface.co (huggingface.co)... 18.164.174.23, 18.164.174.17, 18.164.174.118, ...
Connecting to huggingface.co (huggingface.co)|18.164.174.23|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs-us-1.hf.co/repos/41/c8/41c860f65b01de5dc4c68b00d84cead799d3e7c48e38ee749f4c6057776e2e9e/8a83c7fb9049a9b2e92266fa7ad04933bb53aa1e85136b7b30f1b8000ff2edef?response-content-disposition=inline%3B+filename*%3DUTF-8%27%27Phi-3-mini-4k-instruct-q4.gguf%3B+filename%3D%22Phi-3-mini-4k-instruct-q4.gguf%22%3B&Expires=1739176269&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTczOTE3NjI2OX19LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy11cy0xLmhmLmNvL3JlcG9zLzQxL2M4LzQxYzg2MGY2NWIwMWRlNWRjNGM2OGIwMGQ4NGNlYWQ3OTlkM2U3YzQ4ZTM4ZWU3NDlmNGM2MDU3Nzc2ZTJlOWUvOGE4M2M3ZmI5MDQ5YTliMmU5MjI2NmZhN2FkMDQ5Mz

In [21]:
from langchain import LlamaCpp

# 여러분의 컴퓨터에 다운로드한 모델의 경로를 입력하세요!
llm = LlamaCpp(
    model_path="Phi-3-mini-4k-instruct-q4.gguf",
    n_gpu_layers=-1,
    max_tokens=500,
    n_ctx=4096,
    seed=42,
    verbose=False
)

llama_new_context_with_model: n_batch is less than GGML_KQ_MASK_PAD - increasing to 32


#### 임베딩 모델 로드하기

In [22]:
from langchain.embeddings.huggingface import HuggingFaceEmbeddings

# 텍스트를 수치 표현으로 변환하기 위한 임베딩 모델
embedding_model = HuggingFaceEmbeddings(
    model_name='BAAI/bge-small-en-v1.5'
)

<ipython-input-22-81724f0781e8>:4: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_model = HuggingFaceEmbeddings(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/94.8k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/366 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

#### 벡터 데이터베이스 구축

In [23]:
from langchain.vectorstores import FAISS

# 로컬 벡터 데이터베이스를 만듭니다.
db = FAISS.from_texts(texts, embedding_model)

#### RAG 프롬프트


In [24]:
from langchain import PromptTemplate
from langchain.chains import RetrievalQA


# 프롬프트 템플릿을 만듭니다.
template = """<|user|>
Relevant information:
{context}

Provide a concise answer the following question using the relevant information provided above:
{question}<|end|>
<|assistant|>"""
prompt = PromptTemplate(
    template=template,
    input_variables=["context", "question"]
)

# RAG 파이프라인
rag = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type='stuff',
    retriever=db.as_retriever(),
    chain_type_kwargs={
        "prompt": prompt
    },
    verbose=True
)

In [25]:
rag.invoke('Income generated')



> Entering new RetrievalQA chain...

> Finished chain.


{'query': 'Income generated',
 'result': ' The information provided does not directly relate to the income generated from "Interstellar." However, it mentions that Kip Thorne\'s involvement and the use of special effects by Double Negative could have contributed to the film\'s success. Income generation would depend on various factors like box office earnings, DVD/Blu-ray sales, and additional revenue from tie-in books or related merchandise.'}